Random Forest by Mélusine

In [215]:
# Import
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

In [241]:
# Load datas
data = pd.read_csv("/content/cleaned_merged_seasons.csv", sep=",")

<ipython-input-241-03453092df57>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/cleaned_merged_seasons.csv", sep=",")


In [242]:
print(data.head())

  season_x                name position team_x  assists  bonus  bps  \
0  2016-17     Aaron Cresswell      DEF    NaN        0      0    0   
1  2016-17        Aaron Lennon      MID    NaN        0      0    6   
2  2016-17        Aaron Ramsey      MID    NaN        0      0    5   
3  2016-17  Abdoulaye Doucouré      MID    NaN        0      0    0   
4  2016-17        Adam Forshaw      MID    NaN        0      0    3   

   clean_sheets  creativity  element  ...  team_h_score  threat  total_points  \
0             0         0.0      454  ...           2.0     0.0             0   
1             0         0.3      142  ...           1.0     0.0             1   
2             0         4.9       16  ...           3.0    23.0             2   
3             0         0.0      482  ...           1.0     0.0             0   
4             0         1.3      286  ...           1.0     0.0             1   

   transfers_balance  transfers_in transfers_out  value  was_home  \
0                

In [ ]:
print(data.info())

In [ ]:
print(data.describe())

In [229]:
# valeurs manquantes
data = data.dropna(subset=['team_x'])

# Normalise
features_to_normalize = ["creativity", "ict_index", "influence", "threat"]
data[features_to_normalize] = data[features_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Encode
data_encoded = pd.get_dummies(data, columns=["season_x", "position", "team_x", "kickoff_time", "opp_team_name"], drop_first=True)

# data type int
data_encoded['was_home'] = data_encoded['was_home'].astype(int)

# Retirer la colonne "name"
player_names = data_encoded['name'].reset_index(drop=True)
data_encoded = data_encoded.drop('name', axis=1)

<ipython-input-229-5f571e40d00a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[features_to_normalize] = data[features_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))


In [ ]:
print(player_names)

In [ ]:
print(data_encoded.head())

In [ ]:
print(data_encoded.info())

In [ ]:
print(data_encoded.describe())

In [231]:
# features (X) et cible (y)
X = data_encoded.copy()

# Ajouter les colonnes 'team_x' et 'position' à X
columns_to_add = ['team_x', 'position']
if all(col in data_encoded.columns for col in columns_to_add):
    X[columns_to_add] = data_encoded[columns_to_add]

# Supprimer la colonne 'total_points' de X
X = X.drop('total_points', axis=1)

# Cible (y)
y = data_encoded['total_points']


In [232]:
# Train-test split
X_train, X_test, y_train, y_test, player_names_train, player_names_test = train_test_split(X, y, player_names, test_size=0.2, random_state=42)

In [220]:
print(X_test.columns)

Index(['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'fixture', 'goals_conceded', 'goals_scored', 'ict_index',
       ...
       'opp_team_name_Newcastle', 'opp_team_name_Norwich',
       'opp_team_name_Nott'm Forest', 'opp_team_name_Sheffield Utd',
       'opp_team_name_Southampton', 'opp_team_name_Spurs',
       'opp_team_name_Watford', 'opp_team_name_West Brom',
       'opp_team_name_West Ham', 'opp_team_name_Wolves'],
      dtype='object', length=881)


In [233]:
# Random Forest
random_forest_model = RandomForestRegressor(n_estimators=100, random_state=42)

In [234]:
# Train
random_forest_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [235]:
# Prédictions sur l'ensemble de test
predictions = random_forest_model.predict(X_test)

In [236]:

print(predictions)


[1. 2. 0. ... 0. 0. 5.]


In [237]:
player_names_test = player_names_test.reset_index(drop=True)

In [238]:
results_df = pd.DataFrame({'Name': player_names_test, 'Predicted_Total_Points': predictions})

print(results_df)


                    Name  Predicted_Total_Points
0           Daniel James                     1.0
1           Nicolas Pépé                     2.0
2             Sam Waller                     0.0
3             Bernd Leno                     0.0
4       Ezri Konsa Ngoyo                     6.0
...                  ...                     ...
15259  Aaron Wan-Bissaka                     0.0
15260       Divock Origi                     0.0
15261   Joseph Whitworth                     0.0
15262       Alex Mighten                     0.0
15263  Christian Pulisic                     5.0

[15264 rows x 2 columns]


In [239]:
# performances
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Absolute Error (MAE) on Test Set: {mae}')
print(f'R-squared on Test Set: {r2}')

Mean Absolute Error (MAE) on Test Set: 0.031267033542976934
R-squared on Test Set: 0.9921837594764341


Mean Absolute Error (MAE): Une valeur de MAE aussi basse (0.031) indique que les prédictions du modèle sont en moyenne très proches des valeurs réelles.

R-squared (R2): Une valeur de R-squared aussi élevée (0.992) suggère que votre modèle explique très bien la variance des données de test. En d'autres termes, environ 99.2% de la variance de la variable dépendante (total_points) est expliquée par votre modèle.

In [243]:
from sklearn.model_selection import cross_val_score

# Effectuer la validation croisée
cross_val_scores = cross_val_score(random_forest_model, X, y, cv=5, scoring='neg_mean_absolute_error')

# Calculer la moyenne des scores
average_mae = -cross_val_scores.mean()

# Afficher les résultats
print(f'Average Cross-Validated Mean Absolute Error: {average_mae}')

Average Cross-Validated Mean Absolute Error: 0.027796799839391817


L'obtention d'un "Average Cross-Validated Mean Absolute Error" aussi bas (0.0278) confirme la robustesse de votre modèle, même après avoir été évalué sur plusieurs plis de validation croisée. Une valeur de MAE aussi basse indique que votre modèle est capable de faire des prédictions précises et cohérentes sur des données non vues.

Vous avez donc un modèle qui semble bien généraliser et maintenir sa performance sur différentes portions des données d'entraînement. Cela renforce la confiance dans la capacité du modèle à être efficace lorsqu'il est déployé sur de nouvelles données.